# Test of bound and algorithm

In [1]:
import numpy as np
import gym

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.algorithm.backward_feature_selection import BackwardFeatureSelector
from src.algorithm.feature_selection import Bound
from src.algorithm.info_theory.entropy import LeveOneOutEntropyEstimator, NNEntropyEstimator
from src.wenvs import WrapperEnv
from src.algorithm.utils import episodes_with_len

/home/daibak/.virtualenvs/feature_sel/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
est = NNEntropyEstimator()

In [5]:
env = gym.make('CartPole-v1')
env = WrapperEnv(env, continuous_state=True)
wenv = WrapperEnv(env, n_fake_features=1, n_fake_actions=1, continuous_state=True)
wenv.seed(0)

k = 10
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, k)

In [6]:
fs = BackwardFeatureSelector(est, trajectories)
fs.selectOnError(k, 0.9, 1)

({1, 3, 4, 6}, 0.9100094568123525)

In [8]:
fs.computeError()

0.9100094568123525

## LQG n-dim

In [4]:
from src.envs import lqgNdim

In [14]:
env = lqgNdim.LQG_nD(0.99, n_dim=2)
wenv = WrapperEnv(env, continuous_state=True, continuous_actions=True)

In [15]:
hist = wenv.run_episode(render=True)
wenv.close()

In [16]:
K = env.computeOptimalK()
pi_opt = lambda x: np.clip(K@x, -env.max_action, env.max_action)

In [17]:
hist = wenv.run_episode(policy=pi_opt)
hist[2].sum()

-57.860930209418036

## Infinite CartPole

In [18]:
from src.envs import cartpole

In [19]:
env = cartpole.CartPoleInfinite()
wenv = WrapperEnv(env, continuous_state=True)

In [20]:
wenv.run_episode(render=True)
wenv.close()

## Real test

In [5]:
env = lqgNdim.LQG_nD(0.99, n_dim=2)
wenv = WrapperEnv(env, continuous_state=True, continuous_actions=True)

In [6]:
np.random.seed(0)
wenv.seed(0)

k = 20
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, k)

In [7]:
fs = BackwardFeatureSelector(est, trajectories)

In [8]:
for S, err in fs.try_remove_all(k, 0.9, bound=Bound.cmi):
    print(S, err)

{0, 1, 2} 275.1990224396083


{0, 2} 496.1075058088754


{0} 935.4788314555569


set() 943.514515537091



In [9]:
for S, err in fs.try_remove_all(k, 0.9, bound=Bound.entropy):
    print(S, err)

{0, 1, 2} 275.1990224396083


{0, 2} 496.1075058088752


{0} 935.4788314555569


set() 943.5145155370909



In [10]:
for S, err in fs.try_remove_all(k, 0.9, bound=Bound.cmi_sqrt):
    print(S, err)

{0, 1, 2} 1241.101718300192


{0, 2} 2234.4694059057388


{0} 4168.203698675913


set() 4317.780379381366



In [11]:
Q = np.eye(4) * 0.9
Q[2:,2:] = 0
R = Q.copy()
env = lqgNdim.LQG_nD(0.99, n_dim=4, Q=Q, R=R)
wenv = WrapperEnv(env, continuous_state=True, continuous_actions=True)

In [12]:
np.random.seed(0)
wenv.seed(0)

k = 20
num_ep = 1000
trajectories = episodes_with_len(wenv, num_ep, k)

In [13]:
fs = BackwardFeatureSelector(est, trajectories)

In [14]:
fs.selectNfeatures(1, k, 0.9)

({0}, 985.9146148635155)

In [15]:
for S, err in fs.try_remove_all(k, 0.9):
    print(S, err)

{0, 1, 2, 4, 5, 6, 7} 108.00216084944557


{0, 1, 2, 4, 5, 7} 167.35674911495767


{0, 1, 2, 4, 5} 228.54114149996968


{0, 1, 4, 5} 300.6002932042114


{0, 1, 4} 419.9287663231109


{0, 1} 593.4443694638235


{0} 985.9146148635155


set() 992.1915099307594



In [18]:
for S, err in fs.try_remove_all(k, 0.5):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7} 45.24535799669906


{0, 1, 2, 4, 5, 7} 71.18228218225686


{0, 1, 4, 5, 7} 98.9496011756822


{0, 1, 4, 5} 130.4360593389985


{0, 1, 4} 194.17999425708885


{0, 1} 285.24828747796295


{0} 490.6871943467559


set() 491.31481546994667



In [21]:
for S, err in fs.try_remove_all(k, 0.99):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7} 545.0460961674122


{0, 1, 2, 3, 4, 7} 825.5590212798049


{0, 1, 2, 3, 4} 1098.460291161602


{0, 1, 2, 3} 1388.9583969799105


{0, 1, 3} 1779.6226254877067


{0, 3} 2374.356759976569


{3} 3619.5974031744663


set() 4171.510703275914



In [22]:
for S, err in fs.try_remove_all(k, 0.95):
    print(S, err)

{0, 1, 2, 3, 4, 5, 7} 174.5338703513713


{0, 1, 3, 4, 5, 7} 268.7735621112553


{0, 1, 3, 4, 5} 362.65883506823303


{0, 1, 4, 5} 473.14190819180556


{0, 1, 4} 637.6746542816132


{0, 1} 880.0673789375687


{0} 1425.6404762549394


set() 1484.7182088144207

